In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import math


In [2]:
path = r"data_prueba/prueba_naive_bayes.xlsx"
data = pd.read_excel(path)
data

,Diario,Autor,Fecha,Título,Texto,Vínculo,Clase
0,El Espectador,Nicolás Rodríguez,16 de febrero de 2019,"De Rubio a Acevedo, pasando por Duque",Al presidente Donald Trump poco o nada le inte...,https://web.archive.org/web/20190216230452/htt...,Política
1,El Espectador,Salomón Kalmanovitz,15 de septiembre de 2019,Recentralización,Una de las facetas que acompañan el autoritari...,https://web.archive.org/web/20190916062046/htt...,Política
2,El Espectador,Adriana Cooper,17 de octubre de 2019,La primera alcaldesa de Medellín,Cuando uno revisa los libros sobre historia de...,https://web.archive.org/web/20191018032148/htt...,Política
3,El Espectador,Margarita Flórez,16 de febrero de 2019,"Fracking, luz amarilla a temas de participació...",Basándonos en la presentación realizada esta s...,https://web.archive.org/web/20190217013801/htt...,Medio Ambiente
4,El Espectador,Indalecio Dangond B.,23 de febrero de 2019,La hora del biodiésel,"Primero fue Medellín, ahora le tocó el turno a...",https://web.archive.org/web/20190223102037/htt...,Medio Ambiente
5,El Espectador,Julián López de Mesa Samudio,6 de marzo de 2019,"Biofilia, árboles y ciudad","En 1984, el gran entomólogo, sociobiólogo y do...",https://web.archive.org/web/20190307141807/htt...,Medio Ambiente
6,El Espectador,Santiago Montenegro,11 de febrero de 2019,“Roma”,"Guillermo del Toro, Alejandro González Iñárrit...",https://web.archive.org/web/20190212015339/htt...,Cultura
7,El Espectador,Juan Carlos Botero,14 de junio de 2019,Arte que dura minutos,"La Bienal de Venecia concluyó hace poco, y el ...",https://web.archive.org/web/20190615034748/htt...,Cultura
8,El Espectador,Juan Carlos Gómez,21 de julio de 2019,El futuro de Netflix,El lanzamiento de la tercera temporada de La c...,https://web.archive.org/web/20190722113712/htt...,Cultura
9,El Espectador,Héctor Abad Faciolince,18 de agosto de 2019,Los niños de la guerra,"Lo que más me gusta de esta película, Monos, d...",https://web.archive.org/web/20190818130521/htt...,NaN


# Naive Bayes Normal y Binario
Se seleccionaron 10 columnas, 9 de entrenamiento y una de test. Las de entrenamiento las clasifiqué manualmente en 3 categorías: 3 politica, 3 cultura y 3 medio ambiente. Primero se encuentra el código que entrena un naive bayes básico, y luego se le realiza un cambio para que sea el naive bayes binario.

In [7]:
def train_naive_bayes(D, C):
    # Inicializar estructuras
    logprior = {}
    loglikelihood = {}  
    bigdoc = {}         
    vocabulary = set()

    classes = list(set(C))
    
    # Calcular logprior (P(c))
    Ndoc = len(D)
    for c in classes:
        Nc = sum(1 for clase in C if clase == c)
        logprior[c] = math.log(Nc / Ndoc)
        # Inicializar bigdoc para cada clase
        bigdoc[c] = []
    
    # Construir bigdoc y vocabulario
    for doc, clase in zip(D, C):
        words = doc.split()
        bigdoc[clase].extend(words)  # añade a bigdoc las palabras de la clase (extend aplica a listas)
        vocabulary.update(words)  # añade todas las palabras al vocabulario (update aplica a conjuntos y elimina duplicados)
    
    V = vocabulary
    
    # Calcular loglikelihood (P(w|c))
    for c in classes:
        total_words_in_class = len(bigdoc[c])
        # Inicializar loglikelihood para esta clase
        loglikelihood[c] = {}
        
        for word in V:
            count_wc = bigdoc[c].count(word)
            # Usamos suavizado de Laplace 
            prob = (count_wc + 1) / (total_words_in_class + len(V))
            loglikelihood[c][word] = math.log(prob)
    
    return logprior, loglikelihood, V

def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):
    best_c = None
    max_sum = -float('inf')
    sums = {}
    
    unique_C = set(C)

    # Iterar sobre cada clase única
    for c in unique_C:
        # Inicializar la suma con el logprior de la clase
        sums[c] = logprior[c]
        
        # Procesar cada palabra del documento de prueba
        for word in testdoc.split():
            if word in V and c in loglikelihood and word in loglikelihood[c]:
                sums[c] += loglikelihood[c][word]

        # Actualizar la mejor clase si encontramos una suma mayor
        if sums[c] > max_sum:
            max_sum = sums[c]
            best_c = c  
    
    return best_c, sums


# Datos
documents = data["Texto"].tolist()[:9]  # 9 documentos para entrenamiento
classes = data["Clase"].tolist()[:9]    # clases correspondientes
doc_test = data["Texto"][9]             # documento para prueba

# Entrenamiento
logprior, loglikelihood, V = train_naive_bayes(documents, classes)

# Prueba
predicted_class, probs1 = test_naive_bayes(doc_test, logprior, loglikelihood, classes, V)
print(f"El documento de prueba pertenece a la clase: {predicted_class}")

El documento de prueba pertenece a la clase: Medio Ambiente


# Binario

In [8]:
def train__binary_naive_bayes(D, C):
    # Inicializar estructuras
    logprior = {}
    loglikelihood = {}  
    bigdoc = {}         
    vocabulary = set()

    classes = list(set(C))
    
    # Calcular logprior (P(c))
    Ndoc = len(D)
    for c in classes:
        Nc = sum(1 for clase in C if clase == c)
        logprior[c] = math.log(Nc / Ndoc)
        # Inicializar bigdoc para cada clase
        bigdoc[c] = []
    
    # Construir bigdoc y vocabulario
    for doc, clase in zip(D, C):
        words = set(doc.split())   # EL CAMBIO QUE LO HACE BINARIO
        bigdoc[clase].extend(words)  # añade a bigdoc las palabras de la clase (extend aplica a listas)
        vocabulary.update(words)  # añade todas las palabras al vocabulario (update aplica a conjuntos y elimina duplicados)
    
    V = vocabulary
    
    # Calcular loglikelihood (P(w|c))
    for c in classes:
        total_words_in_class = len(bigdoc[c])
        # Inicializar loglikelihood para esta clase
        loglikelihood[c] = {}
        
        for word in V:
            count_wc = bigdoc[c].count(word)
            # Usamos suavizado de Laplace 
            prob = (count_wc + 1) / (total_words_in_class + len(V))
            loglikelihood[c][word] = math.log(prob)
    
    return logprior, loglikelihood, V

def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):
    best_c = None
    max_sum = -float('inf')
    sums = {}
    
    unique_C = set(C)

    # Iterar sobre cada clase única
    for c in unique_C:
        # Inicializar la suma con el logprior de la clase
        sums[c] = logprior[c]
        
        # Procesar cada palabra del documento de prueba
        for word in testdoc.split():
            if word in V and c in loglikelihood and word in loglikelihood[c]:
                sums[c] += loglikelihood[c][word]

        # Actualizar la mejor clase si encontramos una suma mayor
        if sums[c] > max_sum:
            max_sum = sums[c]
            best_c = c  
    
    return best_c, sums


# Datos
documents = data["Texto"].tolist()[:9]  # 9 documentos para entrenamiento
classes = data["Clase"].tolist()[:9]    # clases correspondientes
doc_test = data["Texto"][9]             # documento para prueba

# Entrenamiento
logprior, loglikelihood, V = train__binary_naive_bayes(documents, classes)

# Prueba
predicted_class, probs2 = test_naive_bayes(doc_test, logprior, loglikelihood, classes, V)
print(f"El documento de prueba pertenece a la clase: {predicted_class}")

El documento de prueba pertenece a la clase: Cultura


In [5]:
loglikelihood

{'Cultura': {'contrarrestar': -7.980023592310645,
  'interna': -7.980023592310645,
  'soltera': -7.980023592310645,
  'fragilidad,': -7.2868764117507,
  'muchos': -7.980023592310645,
  'creían': -7.980023592310645,
  'Hoy': -7.2868764117507,
  'consideraciones': -7.980023592310645,
  'Histórica.': -7.980023592310645,
  'nombres.': -7.980023592310645,
  'artes': -6.881411303642535,
  'técnica.': -7.980023592310645,
  'marciales': -7.2868764117507,
  'universidad': -7.980023592310645,
  'mirada': -7.980023592310645,
  'ahí': -7.980023592310645,
  'contravía': -7.980023592310645,
  'su': -6.593729231190754,
  'conservadores': -7.980023592310645,
  'tendencias': -7.980023592310645,
  'mediano': -7.2868764117507,
  'estuches': -7.2868764117507,
  'pelea': -7.2868764117507,
  'enfatizado,': -7.2868764117507,
  'otros': -7.2868764117507,
  'Rubio,': -7.980023592310645,
  'capital': -7.980023592310645,
  'urgencias.': -7.980023592310645,
  'canchas': -7.980023592310645,
  'Emile': -7.286876411

In [9]:
probs1

{'Cultura': -2099.6537451323657,
 'Medio Ambiente': -2062.2153961809777,
 'Política': -2128.873095182765}

In [10]:
probs2

{'Cultura': -2618.1195261061075,
 'Medio Ambiente': -2638.4935444161306,
 'Política': -2624.1490422543948}